## Libraries

In [51]:
import numpy as np
import pandas as pd
from graph_tool.all import *
import os
import datetime

In [68]:
df = pd.read_csv("data/u2u_6month.csv", index_col = 0)

In [69]:
df

,source,target,time
0,1,91,2008-09-06
1,3,91,2008-09-06
2,380,350,2008-09-06
3,4642,2257,2008-09-06
4,4642,1324220,2008-09-06
...,...,...,...
186711,6521,461736,2008-12-11
186712,20310,18664,2008-11-24
186713,5547,2260,2008-09-10
186714,8553,5556,2008-09-15


## Data Cleaning

In [71]:
## rename columns
df.rename(columns={'source':'src',
                     'target':'tgt',
                     'time':'t'}, inplace = True)

## sort 'ts' into ascending order
df.sort_values('t',ascending=True, inplace=True)

## set index values to 0 
#full.set_index(np.repeat(0, len(full)))

df['t'] = pd.to_datetime(df.t)

In [72]:
df

,src,tgt,t
185852,35,48,2008-08-01
156888,91,48,2008-08-02
156829,91,17,2008-08-02
185754,95,83,2008-08-02
185144,106,71,2008-08-02
...,...,...,...
156388,5023,115,2009-01-31
156389,22704,2603,2009-01-31
156420,48015,115,2009-01-31
156378,45583,40996,2009-01-31


## Time

In [73]:
# fix the timespan for the analysis (in days)
timespan = 30

# time elapsed since the first emailfu
df.loc[:, 'd'] = pd.to_timedelta(df['t'], unit='d')

In [77]:
df['ds'] = df['d'].dt.days

##find days since first day 
df['dt'] = df['ds'] - 14092  

## convert to months 
df.loc[:, 'm'] = df['dt'] // timespan   

In [78]:
df

,src,tgt,t,d,ds,dt,m
185852,35,48,2008-08-01,14092 days,14092,0,0
156888,91,48,2008-08-02,14093 days,14093,1,0
156829,91,17,2008-08-02,14093 days,14093,1,0
185754,95,83,2008-08-02,14093 days,14093,1,0
185144,106,71,2008-08-02,14093 days,14093,1,0
...,...,...,...,...,...,...,...
156388,5023,115,2009-01-31,14275 days,14275,183,6
156389,22704,2603,2009-01-31,14275 days,14275,183,6
156420,48015,115,2009-01-31,14275 days,14275,183,6
156378,45583,40996,2009-01-31,14275 days,14275,183,6


In [83]:
df.drop(["t"], axis = 1, inplace = True)
df.drop(["d"], axis = 1, inplace = True)
df.drop(["ds"], axis = 1, inplace = True)

In [84]:
df

,src,tgt,dt,m
185852,35,48,0,0
156888,91,48,1,0
156829,91,17,1,0
185754,95,83,1,0
185144,106,71,1,0
...,...,...,...,...
156388,5023,115,183,6
156389,22704,2603,183,6
156420,48015,115,183,6
156378,45583,40996,183,6


In [86]:
# sources
n = list(set(df['src']))

# targets
n.extend(list(set(df['tgt'])))

# collapse sources and targets
n = list(set(n))

# get a data frame whose index is constant and equal to zero
n = pd.DataFrame({'n': n}, index=np.repeat(0, len(n)))

In [85]:
# unique months
m = list(set(df['m']))

# get a data frame whose index is constant and equal to zero
m = pd.DataFrame({'m': m}, index=np.repeat(0, len(m)))

In [88]:
rs = pd.merge(n, n, left_index=True, right_index=True)

# risk set-timespan cartesian product
#rs = pd.merge(rs, m, left_index=True, right_index=True)
       
# focus on the lower triangle of the adj matrix
#rs = rs.loc[rs['n_x'] < rs['n_y']]

MemoryError: Unable to allocate array with shape (258984649,) and data type int64